In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Filtering all the warnings sign

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False # if autocompletion doesnot work in kaggle notebook | hit tab

#### Loading the data 

In [ ]:
train_df = pd.read_csv('../input/nlp-getting-started/train.csv',usecols=['id','text','target'])
test_df = pd.read_csv('../input/nlp-getting-started/test.csv',usecols=['id','text'])
sample_df = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

here sample_df shows up the format of data to be uploaded on kaggle comptetions

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
train_df.shape

#### We need to do some text cleaning
here some signs and characters need to be removed , again cleaning the text data before training is a good practice well bert is more advanced architecture , it doesn't much affect if we dont do data cleaning 
, bert dont need extensive text_cleaning because bert comes with 40/60000 words hence its really not necessary to do text_cleaning but removing the special characters are good practice

In [ ]:
!pip install text_hammer 

In [ ]:
import text_hammer as th

In [ ]:
%%time

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
#     df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
#     df[column] = df[column].progress_apply(lambda x: ps.remove_stopwords(x))

    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
#     df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    return(df)

In [ ]:
train_cleaned_df = text_preprocessing(train_df,'text')


In [ ]:
train_cleaned_df[train_cleaned_df.target == 0]

In [ ]:
train_df = train_cleaned_df.copy()

here target 1 means we are talking about any accident or disaster and 0 means just a formal tweets with not much attention

so far we have cleaned our text data and now lets load our model

#### Loading Pretrained BERT Model

In [ ]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
bert = TFBertModel.from_pretrained('bert-large-uncased')

In [ ]:
tokenizer('this is me abhishek and i am a very bad boy &*&*&&')

#### Convert Our text data into BERT input format 

In [ ]:
print("max len of tweets",max([len(x.split()) for x in train_df.text]))
max_length = 36

In [ ]:
x_train = tokenizer(
    text=train_df.text.tolist(),
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


In [ ]:
x_train['input_ids'].shape

In [ ]:
x_train['attention_mask'].shape

In [ ]:
y_train = train_df.target.values
y_train

In [ ]:
train_df.target.value_counts()

#### Building the model architecture 

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [ ]:
max_len = 36
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[1] #(0 is the last hidden states,1 means pooler_output)
# out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = tf.keras.layers.Dropout(0.1)(embeddings)

out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(1,activation = 'sigmoid')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
# for training bert our lr must be so small

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(
    learning_rate=6e-06, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = BinaryCrossentropy(from_logits = True)
metric = BinaryAccuracy('accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
#     validation_split = 0.1,
  epochs=10,
    batch_size=10
)

#### TESTING PHASE
on this phase we will make predictions out of our model and then submit to kaggle comptetions

In [ ]:
test_df

In [ ]:
# test_df = text_preprocessing(test_df,'text')
# without cleaning the model gives better accuracy on uplaoding 

In [ ]:
x_test = tokenizer(
    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


In [ ]:
predicted = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [ ]:
y_predicted = np.where(predicted>0.5,1,0)

In [ ]:
y_predicted = y_predicted.reshape((1,3263))[0]

In [ ]:
sample_df['id'] = test_df.id
sample_df['target'] = y_predicted

In [ ]:
sample_df.to_csv('submission4.csv',index = False)